## Выбор модели для задач классификации

Для задач классификации существует несколько способов выбора модели:

1. **Обучить маленькую модель с нуля**: Выбор легковесной архитектуры модели и полное обучение её на нашем наборе данных.
2. **Fine-Tuning предобученной модели**: Используем модель, которая была предобучена авторами на другом наборе данных, и только последние слои переобучается на нашем конкретном наборе данных.
3. **Использовать предобученные веса напрямую**: В этом методе используется предобученная модель без дополнительного обучения.

Лучшим вариантом является Fine-Tuning, поэтому в качестве бейзлайна возьмем маленькую (5.3M) EfficientNet B0 с претрейновыми параметрами

[Оригинальная статья](https://arxiv.org/abs/1905.11946)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

In [2]:
import os
import zipfile

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
local_zip = "/content/drive/MyDrive/dogvscat_small.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("/content/dataset")
zip_ref.close()

In [5]:
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # Размер входа в сеть
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # Нормализация
    ]
)

train_dataset = datasets.ImageFolder(
    "/content/dataset/dogvscat_small/train", transform=transform
)
test_dataset = datasets.ImageFolder(
    "/content/dataset/dogvscat_small/test", transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Проверка классов
print(train_dataset.class_to_idx)  # {'dog': 0, 'cat': 1}

{'cats': 0, 'dogs': 1}


In [9]:
# Загрузка модели EfficientNet
model = efficientnet_b0(EfficientNet_B0_Weights.DEFAULT)

# Заморозка всех слоев, кроме последнего
for param in model.parameters():
    param.requires_grad = False

# Замена финального классификатора
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, 128),
    nn.ReLU(),
    nn.Linear(128, 1),  # Бинарная классификация
    nn.Sigmoid(),  # Для вероятностей
)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 94.3MB/s]


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCELoss()  # Для бинарной классификации
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

In [11]:
num_epochs = 10

for epoch in range(num_epochs):
    # Тренировка
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Валидация
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()

            outputs = model(images).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    accuracy = correct / total

    print(
        f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}"
    )

Epoch 1/10, Train Loss: 0.6808, Val Loss: 0.6583, Accuracy: 0.7830
Epoch 2/10, Train Loss: 0.6396, Val Loss: 0.6263, Accuracy: 0.8770
Epoch 3/10, Train Loss: 0.5779, Val Loss: 0.5874, Accuracy: 0.8930
Epoch 4/10, Train Loss: 0.5548, Val Loss: 0.5487, Accuracy: 0.8960
Epoch 5/10, Train Loss: 0.4854, Val Loss: 0.5046, Accuracy: 0.9050
Epoch 6/10, Train Loss: 0.4187, Val Loss: 0.4675, Accuracy: 0.9080
Epoch 7/10, Train Loss: 0.4038, Val Loss: 0.4354, Accuracy: 0.9090
Epoch 8/10, Train Loss: 0.3512, Val Loss: 0.4019, Accuracy: 0.9050
Epoch 9/10, Train Loss: 0.3315, Val Loss: 0.3848, Accuracy: 0.9110
Epoch 10/10, Train Loss: 0.2941, Val Loss: 0.3569, Accuracy: 0.9170
